#1.Dataset  
load the AG News dataset, rename columns, and combine the title and description into a single text field for processing.


In [3]:
import pandas as pd

df = pd.read_csv('train.csv', header=None)
df.columns = ['label', 'title', 'description']
df['text'] = df['title'] + ' ' + df['description']
df = df[['text', 'label']]
df.head()

text        label
0                                  Title Description  Class Index
1  Wall St. Bears Claw Back Into the Black (Reute...            3
2  Carlyle Looks Toward Commercial Aerospace (Reu...            3
3  Oil and Economy Cloud Stocks' Outlook (Reuters...            3
4  Iraq Halts Oil Exports from Main Southern Pipe...            3

#2.Clean Data  
removes URLs, special characters,stopwords.clean & normalized text for vectorization.


In [4]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = str(text)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['clean_text'] = df['text'].apply(clean_text)
df.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


text        label  \
0                                  Title Description  Class Index   
1  Wall St. Bears Claw Back Into the Black (Reute...            3   
2  Carlyle Looks Toward Commercial Aerospace (Reu...            3   
3  Oil and Economy Cloud Stocks' Outlook (Reuters...            3   
4  Iraq Halts Oil Exports from Main Southern Pipe...            3   

                                          clean_text  
0                                  title description  
1  wall st bears claw back black reuters reuters ...  
2  carlyle looks toward commercial aerospace reut...  
3  oil economy cloud stocks outlook reuters reute...  
4  iraq halts oil exports main southern pipeline ...

#3.Convert text to numeric features (TF-IDF)  
use TF-IDF vectorization to convert cleaned text to numerical features to model can understand.


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(df['clean_text'])
y = df['label']

#4.Train classification model  
train (Logistic Regression) model to classify the news topics based on the TF-IDF features.

In [7]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X, y)

LogisticRegression(max_iter=1000)

#5. Evaluate model  
split the data to training and testing sets, evaluate the model using accuracy score and a confusion matrix.

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", acc)
print("Confusion Matrix:\n", cm)

Accuracy: 0.9156285154785218
Confusion Matrix:
 [[5450  160  248  189    0]
 [  55 5759   32   26    0]
 [ 195   67 5339  478    0]
 [ 173   49  352 5428    0]
 [   0    1    0    0    0]]


In [13]:
label_html = {
    1: '<div style="display:flex;align-items:center;"><img src="https://cdn-icons-png.flaticon.com/512/219/219983.png" width="25" style="margin-right:8px;">World</div>',
    2: '<div style="display:flex;align-items:center;"><img src="https://cdn-icons-png.flaticon.com/512/905/905568.png" width="25" style="margin-right:8px;">Sports</div>',
    3: '<div style="display:flex;align-items:center;"><img src="https://cdn-icons-png.flaticon.com/512/3135/3135715.png" width="25" style="margin-right:8px;">Business</div>',
    4: '<div style="display:flex;align-items:center;"><img src="https://cdn-icons-png.flaticon.com/512/2965/2965567.png" width="25" style="margin-right:8px;">Science & Tech</div>'
}

In [10]:
!pip install gradio

#6.Bulid Gradio App  



In [18]:
import gradio as gr

def predict_topic_display(text):
    cleaned = clean_text(text)
    vectorized = vectorizer.transform([cleaned])
    pred = model.predict(vectorized)[0]
    return label_html.get(int(pred), "Unknown")

gr.Interface(
    fn=predict_topic_display,
    inputs=gr.Textbox(lines=4, placeholder="Enter a news headline or paragraph...", label="News Text"),
    outputs=gr.HTML(label="Predicted Topic"),
    title="News Topic Classifier",
    description="Enter a short news article or title to identify its category.",
    theme="default"
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://01fea7690774303ca7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
